In [1]:
import pandas as pd 
import numpy as np

In [2]:

data=pd.read_csv(r"C:\Users\CG-DTE\Desktop\AI\GITHUB\Predicting bankruptcy\low level model analysis\5year.csv")

### above feature set i have gone through some feature engineering steps
#### missing imputation------>gaussian transformation ----->decision tree descritisation------>scaling --->feature selection
#### [['Attr1', 'Attr4', 'Attr21', 'Attr27', 'Attr34', 'Attr35', 'Attr52', 'Attr53', 'Attr58']] i got these features

### but important analysis i found that feature set using these above feature engineering steps giving best result on original data aftter imputation only

In [3]:
year_4=data[['Attr1', 'Attr4', 'Attr21', 'Attr27', 'Attr34', 'Attr35', 'Attr52', 'Attr53', 'Attr58','class']]

In [4]:
k1=year_4.columns.tolist()

In [5]:
#replacing each ? to NAN
for i in k1:
    year_4[i] = year_4[i].replace('?' , np.nan)

<ipython-input-5-763fa34d3bd1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_4[i] = year_4[i].replace('?' , np.nan)


In [6]:
#converting object data into numericals(float64)
year_4=year_4.apply(lambda col:pd.to_numeric(col,errors='coerce'))

In [7]:
year_4

,Attr1,Attr4,Attr21,Attr27,Attr34,Attr35,Attr52,Attr53,Attr58,class
0,0.088238,1.02050,1.15740,1.03870,0.24377,0.135230,0.42557,0.73717,0.91905,0
1,-0.006202,1.59980,1.01580,0.17118,2.70750,-0.036475,0.29604,1.36140,1.00470,0
2,0.130240,3.60820,1.23620,1.47370,0.65878,0.145860,0.22371,3.36840,0.87604,0
3,-0.089951,1.52220,1.09420,0.00000,0.83567,0.014027,0.69565,0.52538,0.59074,0
4,0.048179,1.24370,0.99455,2.06710,2.13360,0.364200,0.37618,0.99779,0.77048,0
...,...,...,...,...,...,...,...,...,...,...
5905,0.012898,1.17220,1.03300,1.46370,2.36440,0.033819,0.13514,0.39944,1.01220,1
5906,-0.578050,0.16576,0.64770,NaN,-0.55316,-0.534920,0.20912,-0.46385,1.06410,1
5907,-0.179050,0.74554,0.83104,NaN,0.14077,0.176700,0.66913,-1.70670,0.85112,1
5908,-0.108860,1.08780,1.12100,NaN,-0.13435,-0.099950,0.19247,0.11530,1.18320,1


### so here i m building model on entire dataset for , so i m not divinding into train and test dataset , so that i do have ample amount of data to train our model more accurately

In [8]:
predictors=['Attr1', 'Attr4', 'Attr21', 'Attr27', 'Attr34', 'Attr35', 'Attr52', 'Attr53', 'Attr58']
target=['class']
X=year_4[predictors]
y=year_4[target]

In [9]:
#X
#y

In [10]:
from sklearn.impute import SimpleImputer

In [11]:
# Now we impute the missing values with SimpleImputer

# create an instance of the simple imputer
# we indicate that we want to impute with the median
mean_imputer = SimpleImputer(strategy='mean',)

# we fit and transform the predictors dataframe
# the imputer will learn the mean of all variables
X_new=mean_imputer.fit_transform(X[predictors])

In [12]:
y_1=y.copy()
X_1=pd.DataFrame(X_new, columns=predictors)

### over sampling through SMOTE is performed

In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2) 
X_res, y_res = sm.fit_resample(X_1, y_1)

In [14]:
len(X_res)

11000

In [15]:

len(y_res)

11000

### Note:- since , i have not splited my data into train-test dataset so my independent and dependent data are in pandas dataframe so in order to implement in XGBoostingClassifier algo in pickled , model will not take dataframe data as input to predict , so i need to convert X(input data ) into numpy array , i m not converting y into numpy array¶


In [16]:
#type(X_res)
type(y_res)

pandas.core.frame.DataFrame

In [17]:
X=np.array(X_res)
y=np.array(y_res)

In [18]:
#type(X)
#type(y)

In [19]:
import seaborn as sns
import xgboost as xgb
# Classification metrices
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report, precision_score,recall_score,f1_score

In [20]:
xgb.__version__

'1.5.0'

### functionalizing the model

In [21]:

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import pickle

In [22]:
# Functionalize model fittting

def FitModel(X,Y,algo_name,algorithm,gridSearchParams,cv):
    np.random.seed(10)
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2)
    
    
    grid = GridSearchCV(
        estimator=algorithm,
        param_grid=gridSearchParams,
        cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
    
    
    grid_result = grid.fit(x_train, y_train)
    best_params = grid_result.best_params_
    pred = grid_result.predict(x_test)
    cm = confusion_matrix(y_test, pred)
   # metrics =grid_result.gr
    print(pred)
    pickle.dump(grid_result,open(algo_name,'wb'))
   
    print('Best Params :',best_params)
    print('Classification Report :',classification_report(y_test,pred))
    print('Accuracy Score : ' + str(accuracy_score(y_test,pred)))
    print('Confusion Matrix : \n', cm)

In [23]:
param ={
            'n_estimators': [50, 100, 500, 1000, 2000],
           
        }
FitModel(X,y,'finalized_model1.pkl',XGBClassifier(),param,cv=5)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\CG-DTE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\CG-DTE\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\CG-DTE\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[18:58:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0 1 1 ... 0 1 0]
Best Params : {'n_estimators': 500}
Classification Report :               precision    recall  f1-score   support

           0       0.98      0.95      0.97      1139
           1       0.95      0.98      0.97      1061

    accuracy                           0.97      2200
   macro avg       0.97      0.97      0.97      2200
weighted avg       0.97      0.97      0.97      2200

Accuracy Score : 0.9663636363636363
Confusion Matrix : 
 [[1086   53]
 [  21 1040]]


### Reloading the Saved Model

In [24]:
loaded_model = pickle.load(open("finalized_model1.pkl","rb"))

In [25]:
pred = loaded_model.predict(np.array([[0.035999,1.55860,1.32360,0.88616,0.24767,0.084147,0.297850,1.07620,0.90850]]))

In [26]:

print(pred)

[0]


In [27]:
def change (prediction):
    if prediction==1:
        return 'yes'
    else:
        return 'No'

In [28]:
print('some of the features of company:-')
print('net profit / total assets = 0.035999')
print('current assets / short-term liabilities = 1.55860 ')
print('sales (n) / sales (n-1) = 1.32360')
print('profit on operating activities / financial expenses = 0.88616')
print('operating expenses / total liabilities = 0.24767')
print('profit on sales / total assets = 0.084147')
print('(short-term liabilities * 365) / cost of products sold) = 0.297850')
print('equity / fixed assets = 1.07620')
print('total costs /total sales = 0.90850')

print('\n\n')

print('will company go for bankruptcy ? {}'.format(change(pred)))

some of the features of company:-
net profit / total assets = 0.035999
current assets / short-term liabilities = 1.55860 
sales (n) / sales (n-1) = 1.32360
profit on operating activities / financial expenses = 0.88616
operating expenses / total liabilities = 0.24767
profit on sales / total assets = 0.084147
(short-term liabilities * 365) / cost of products sold) = 0.297850
equity / fixed assets = 1.07620
total costs /total sales = 0.90850



will company go for bankruptcy ? No
